In [ ]:
#Q-1

In [7]:
%%writefile sum_tasks_debug.cu
#include <iostream>
#include <cuda_runtime.h>

#define N 1024

__global__ void sumTasks(int* output) {
    int tid = threadIdx.x;

    if (tid == 0) {
        printf("Thread 0 running (iterative sum)...\n");
        int sum = 0;
        for (int i = 1; i <= N; i++) {
            sum += i;
        }
        output[0] = sum;
    }
    else if (tid == 1) {
        printf("Thread 1 running (formula sum)...\n");
        output[1] = N * (N + 1) / 2;
    }
}

void checkCudaError(cudaError_t err, const char* msg) {
    if (err != cudaSuccess) {
        std::cerr << "CUDA Error: " << msg << " -> " << cudaGetErrorString(err) << std::endl;
        exit(EXIT_FAILURE);
    }
}

int main() {
    int h_output[2] = {0, 0};
    int* d_output;

    // Allocate memory
    checkCudaError(cudaMalloc((void**)&d_output, 2 * sizeof(int)), "Allocating device memory");

    // Initialize device memory to 0
    checkCudaError(cudaMemset(d_output, 0, 2 * sizeof(int)), "Memset device memory");

    // Launch kernel
    sumTasks<<<1, 2>>>(d_output);

    // Check kernel launch error
    checkCudaError(cudaGetLastError(), "Launching kernel");

    // Sync to ensure it's done
    checkCudaError(cudaDeviceSynchronize(), "Synchronizing device");

    // Copy result back
    checkCudaError(cudaMemcpy(h_output, d_output, 2 * sizeof(int), cudaMemcpyDeviceToHost), "Copying back results");

    std::cout << "Iterative sum (Thread 0): " << h_output[0] << std::endl;
    std::cout << "Formula sum (Thread 1): " << h_output[1] << std::endl;

    cudaFree(d_output);
    return 0;
}


Writing sum_tasks_debug.cu


In [12]:
!nvcc -arch=sm_75 sum_tasks_debug.cu -o sum_tasks_debug

In [13]:
!./sum_tasks_debug

Thread 1 running (formula sum)...
Thread 0 running (iterative sum)...
Iterative sum (Thread 0): 524800
Formula sum (Thread 1): 524800


In [14]:
#Q-2(a)CPU pipelined merge sort

In [15]:
%%writefile pipelined_merge_sort.cpp
#include <iostream>
#include <vector>
#include <omp.h>
#include <chrono>
#include <algorithm>

#define N 1000

void merge(std::vector<int>& arr, int left, int mid, int right) {
    std::vector<int> leftPart(arr.begin() + left, arr.begin() + mid + 1);
    std::vector<int> rightPart(arr.begin() + mid + 1, arr.begin() + right + 1);

    int i = 0, j = 0, k = left;
    while (i < leftPart.size() && j < rightPart.size()) {
        if (leftPart[i] < rightPart[j]) arr[k++] = leftPart[i++];
        else arr[k++] = rightPart[j++];
    }

    while (i < leftPart.size()) arr[k++] = leftPart[i++];
    while (j < rightPart.size()) arr[k++] = rightPart[j++];
}

void parallelMergeSort(std::vector<int>& arr, int left, int right, int depth = 0) {
    if (left >= right) return;
    int mid = (left + right) / 2;

    if (depth < 3) {
        #pragma omp parallel sections
        {
            #pragma omp section
            parallelMergeSort(arr, left, mid, depth + 1);

            #pragma omp section
            parallelMergeSort(arr, mid + 1, right, depth + 1);
        }
    } else {
        parallelMergeSort(arr, left, mid, depth + 1);
        parallelMergeSort(arr, mid + 1, right, depth + 1);
    }

    merge(arr, left, mid, right);
}

int main() {
    std::vector<int> arr(N);
    for (int i = 0; i < N; ++i)
        arr[i] = rand() % 1000;

    auto start = std::chrono::high_resolution_clock::now();
    parallelMergeSort(arr, 0, N - 1);
    auto end = std::chrono::high_resolution_clock::now();

    double time = std::chrono::duration<double, std::milli>(end - start).count();
    std::cout << "CPU Parallel (Pipelined) Merge Sort Time: " << time << " ms" << std::endl;

    return 0;
}


Writing pipelined_merge_sort.cpp


In [16]:
!g++ -fopenmp pipelined_merge_sort.cpp -o pipelined_merge_sort

In [17]:
!./pipelined_merge_sort

CPU Parallel (Pipelined) Merge Sort Time: 0.693341 ms


In [18]:
#(b)
%%writefile cuda_merge_sort.cu
#include <iostream>
#include <cuda.h>
#include <stdlib.h>
#include <chrono>

#define N 1000

__device__ void merge(int* input, int* temp, int left, int mid, int right) {
    int i = left, j = mid + 1, k = left;
    while (i <= mid && j <= right) {
        if (input[i] <= input[j]) temp[k++] = input[i++];
        else temp[k++] = input[j++];
    }
    while (i <= mid) temp[k++] = input[i++];
    while (j <= right) temp[k++] = input[j++];
    for (int l = left; l <= right; l++) input[l] = temp[l];
}

__global__ void mergeSortKernel(int* input, int* temp, int width, int size) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int left = tid * width * 2;
    if (left + width < size) {
        int mid = left + width - 1;
        int right = min(left + 2 * width - 1, size - 1);
        merge(input, temp, left, mid, right);
    }
}

void mergeSortGPU(int* h_input) {
    int* d_input;
    int* d_temp;

    cudaMalloc(&d_input, N * sizeof(int));
    cudaMalloc(&d_temp, N * sizeof(int));
    cudaMemcpy(d_input, h_input, N * sizeof(int), cudaMemcpyHostToDevice);

    for (int width = 1; width < N; width *= 2) {
        int numBlocks = (N / (2 * width)) + 1;
        mergeSortKernel<<<numBlocks, 1>>>(d_input, d_temp, width, N);
        cudaDeviceSynchronize();
    }

    cudaMemcpy(h_input, d_input, N * sizeof(int), cudaMemcpyDeviceToHost);
    cudaFree(d_input);
    cudaFree(d_temp);
}

int main() {
    int* arr = new int[N];
    for (int i = 0; i < N; ++i)
        arr[i] = rand() % 1000;

    auto start = std::chrono::high_resolution_clock::now();
    mergeSortGPU(arr);
    auto end = std::chrono::high_resolution_clock::now();

    double time = std::chrono::duration<double, std::milli>(end - start).count();
    std::cout << "CUDA Merge Sort Time: " << time << " ms" << std::endl;

    delete[] arr;
    return 0;
}


Writing cuda_merge_sort.cu


In [19]:
!nvcc -arch=sm_75 cuda_merge_sort.cu -o cuda_merge_sort
!./cuda_merge_sort

CUDA Merge Sort Time: 200.04 ms
